In [1]:
# Import libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import psycopg2
import getpass
import pdvega
import seaborn as sns
# for configuring connection 
from configobj import ConfigObj
import os

%matplotlib inline

In [2]:
# Create a database connection using settings from config file
config='../db/config.ini'

# connection info
conn_info = dict()
if os.path.isfile(config):
    config = ConfigObj(config)
    conn_info["sqluser"] = config['username']
    conn_info["sqlpass"] = config['password']
    conn_info["sqlhost"] = config['host']
    conn_info["sqlport"] = config['port']
    conn_info["dbname"] = config['dbname']
    conn_info["schema_name"] = config['schema_name']
else:
    conn_info["sqluser"] = 'postgres'
    conn_info["sqlpass"] = ''
    conn_info["sqlhost"] = '192.168.60.144'
    conn_info["sqlport"] = 6432
    conn_info["dbname"] = 'eicu'
    conn_info["schema_name"] = 'public,eicu_crd'
    
# Connect to the eICU database
print('Database: {}'.format(conn_info['dbname']))
print('Username: {}'.format(conn_info["sqluser"]))
if conn_info["sqlpass"] == '':
    # try connecting without password, i.e. peer or OS authentication
    try:
        if (conn_info["sqlhost"] == '192.168.60.144') & (conn_info["sqlport"]=='6432'):
            con = psycopg2.connect(dbname=conn_info["dbname"],
                                   user=conn_info["sqluser"])            
        else:
            con = psycopg2.connect(dbname=conn_info["dbname"],
                                   host=conn_info["sqlhost"],
                                   port=conn_info["sqlport"],
                                   user=conn_info["sqluser"])
    except:
        conn_info["sqlpass"] = getpass.getpass('Password: ')

        con = psycopg2.connect(dbname=conn_info["dbname"],
                               host=conn_info["sqlhost"],
                               port=conn_info["sqlport"],
                               user=conn_info["sqluser"],
                               password=conn_info["sqlpass"])
query_schema = 'set search_path to ' + conn_info['schema_name'] + ';'

Database: eicu
Username: postgres


In [5]:
from sqlalchemy import create_engine,text
con= create_engine('postgresql://eicu@192.168.60.144:6432/eicu')

In [6]:
datadir = '/home/mei/nas/docker/dataset/EICU/eicu-collaborative-research-database-2.0/'
porcesseddir2 = '/home/mei/nas/docker/processedData_2/'

In [7]:
def round_up(x, base=5):
    return base * round(x/base)

In [8]:
query = query_schema + """
select p.patientunitstayid, p.hospitaladmitOffset,l.labresultoffset,p.gender, p.age, 
      p.apacheadmissiondx,l.labname,l.labresult,p.unitadmitsource, p.admissionweight,	
      p.dischargeweight, p.unitdischargeoffset, p.unitdischargelocation,	
      p.unitdischargestatus
from lab l
join patient_2 p
  on l.patientunitstayid = p.patientunitstayid
  
order by p.patientunitstayid, p.hospitaladmitOffset,l.labresultoffset
"""

In [9]:
df_lab= pd.read_sql_query(query,con)
df_lab.head(n=20)

,patientunitstayid,hospitaladmitoffset,labresultoffset,gender,age,apacheadmissiondx,labname,labresult,unitadmitsource,admissionweight,dischargeweight,unitdischargeoffset,unitdischargelocation,unitdischargestatus
0,141168,0,231,Female,70,"Rhythm disturbance (atrial, supraventricular)",PT - INR,1.70,Direct Admit,84.3,85.8,3596,Death,Expired
1,141168,0,231,Female,70,"Rhythm disturbance (atrial, supraventricular)",PT,17.10,Direct Admit,84.3,85.8,3596,Death,Expired
2,141168,0,231,Female,70,"Rhythm disturbance (atrial, supraventricular)",PTT,29.00,Direct Admit,84.3,85.8,3596,Death,Expired
3,141168,0,516,Female,70,"Rhythm disturbance (atrial, supraventricular)",WBC x 1000,9.80,Direct Admit,84.3,85.8,3596,Death,Expired
4,141168,0,516,Female,70,"Rhythm disturbance (atrial, supraventricular)",albumin,3.10,Direct Admit,84.3,85.8,3596,Death,Expired
5,141168,0,516,Female,70,"Rhythm disturbance (atrial, supraventricular)",bicarbonate,26.00,Direct Admit,84.3,85.8,3596,Death,Expired
6,141168,0,516,Female,70,"Rhythm disturbance (atrial, supraventricular)",-eos,1.00,Direct Admit,84.3,85.8,3596,Death,Expired
7,141168,0,516,Female,70,"Rhythm disturbance (atrial, supraventricular)",total protein,7.10,Direct Admit,84.3,85.8,3596,Death,Expired
8,141168,0,516,Female,70,"Rhythm disturbance (atrial, supraventricular)",BUN,26.00,Direct Admit,84.3,85.8,3596,Death,Expired
9,141168,0,516,Female,70,"Rhythm disturbance (atrial, supraventricular)",PT - INR,1.90,Direct Admit,84.3,85.8,3596,Death,Expired


In [10]:
lab=df_lab.copy()
lab.set_index(['patientunitstayid', 'hospitaladmitoffset','labresultoffset'], inplace = True)
lab.drop(lab.index[np.where(lab.index.get_level_values('labresultoffset') < 0)], inplace= True)
# Pivot table
lab = pd.pivot_table(lab ,values = 'labresult', index=['patientunitstayid', 'hospitaladmitoffset','labresultoffset'], columns='labname')
lab.columns.name = None
# Resample every 5 mins
lab.rename(round_up, level = 'labresultoffset', inplace = True)
# lab = lab.groupby(['patientunitstayid', 'hospitaladmitoffset','labresultoffset']).mean()
lab.head()


-bands  -basos  -eos  \
patientunitstayid hospitaladmitoffset labresultoffset                         
141168            0                   230                 NaN     NaN   NaN   
                                      515                 NaN     0.0   1.0   
                                      1135                NaN     0.0   0.0   
                                      1135                NaN     NaN   NaN   
                                      1700                NaN     NaN   NaN   

                                                       -lymphs  -monos  \
patientunitstayid hospitaladmitoffset labresultoffset                    
141168            0                   230                  NaN     NaN   
                                      515                 19.0    19.0   
                                      1135                 6.0    14.0   
                                      1135                 NaN     NaN   
                                      1700                 NaN     NaN   

                                                       -polys  \
patientunitstayid hospitaladmitoffset labresultoffset           
141168            0                   230                 NaN   
                                      515                61.0   
                                      1135               80.0   
                                      1135                NaN   
                                      1700                NaN   

                                                       24 h urine protein  \
patientunitstayid hospitaladmitoffset labresultoffset                       
141168            0                   230                             NaN   
                                      515                             NaN   
                                      1135                            NaN   
                                      1135                            NaN   
                                      1700                            NaN   

                                                       24 h urine urea nitrogen  \
patientunitstayid hospitaladmitoffset labresultoffset                             
141168            0                   230                                   NaN   
                                      515                                   NaN   
                                      1135                                  NaN   
                                      1135                                  NaN   
                                      1700                                  NaN   

                                                       ALT (SGPT)  ANF/ANA  \
patientunitstayid hospitaladmitoffset labresultoffset                        
141168            0                   230                     NaN      NaN   
                                      515                    40.0      NaN   
                                      1135                  358.0      NaN   
                                      1135                    NaN      NaN   
                                      1700                    NaN      NaN   

                                                       ...  total protein  \
patientunitstayid hospitaladmitoffset labresultoffset  ...                  
141168            0                   230              ...            NaN   
                                      515              ...            7.1   
                                      1135             ...            7.1   
                                      1135             ...            NaN   
                                      1700             ...            NaN   

                                                       transferrin  \
patientunitstayid hospitaladmitoffset labresultoffset                
141168            0                   230                      NaN   
                                      515                      NaN   
                                     

In [11]:
lab_ = lab.groupby(['patientunitstayid', 'hospitaladmitoffset','labresultoffset']).mean()
lab_.head()

-bands  -basos  -eos  \
patientunitstayid hospitaladmitoffset labresultoffset                         
141168            0                   230                 NaN     NaN   NaN   
                                      515                 NaN     0.0   1.0   
                                      1135                NaN     0.0   0.0   
                                      1700                NaN     NaN   NaN   
                                      1805                NaN     NaN   NaN   

                                                       -lymphs  -monos  \
patientunitstayid hospitaladmitoffset labresultoffset                    
141168            0                   230                  NaN     NaN   
                                      515                 19.0    19.0   
                                      1135                 6.0    14.0   
                                      1700                 NaN     NaN   
                                      1805                 NaN     NaN   

                                                       -polys  \
patientunitstayid hospitaladmitoffset labresultoffset           
141168            0                   230                 NaN   
                                      515                61.0   
                                      1135               80.0   
                                      1700                NaN   
                                      1805                NaN   

                                                       24 h urine protein  \
patientunitstayid hospitaladmitoffset labresultoffset                       
141168            0                   230                             NaN   
                                      515                             NaN   
                                      1135                            NaN   
                                      1700                            NaN   
                                      1805                            NaN   

                                                       24 h urine urea nitrogen  \
patientunitstayid hospitaladmitoffset labresultoffset                             
141168            0                   230                                   NaN   
                                      515                                   NaN   
                                      1135                                  NaN   
                                      1700                                  NaN   
                                      1805                                  NaN   

                                                       ALT (SGPT)  ANF/ANA  \
patientunitstayid hospitaladmitoffset labresultoffset                        
141168            0                   230                     NaN      NaN   
                                      515                    40.0      NaN   
                                      1135                  358.0      NaN   
                                      1700                    NaN      NaN   
                                      1805                    NaN      NaN   

                                                       ...  total protein  \
patientunitstayid hospitaladmitoffset labresultoffset  ...                  
141168            0                   230              ...            NaN   
                                      515              ...            7.1   
                                      1135             ...            7.1   
                                      1700             ...            NaN   
                                      1805             ...            NaN   

                                                       transferrin  \
patientunitstayid hospitaladmitoffset labresultoffset                
141168            0                   230                      NaN   
                                      515                      NaN   
                                     